In [1]:
#files
import sys
#from os import listdir
sys.path.append('../data')
sys.path.append('../Librerias')
#from os.path import isfile, join

#own libs
import utils
import caract as dc
import dataset as ds
from models import SimpleRNN, LSTM, MLP
#from lstm_models import LSTM
import utils_2 as ut2
from traindata import trainData
import errores as er



#data and visualization
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, root_mean_squared_error, r2_score

sns.set_style("darkgrid")


import numpy as np
import random
import os, errno
from tqdm import tqdm

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as data_utils


np.random.seed(seed=77)
torch.manual_seed(77)

In [2]:
#fpath = "/Users/consu/OneDrive/Documentos/GitHub/Tesis/data/low/"
#fpath = "/Users/granosftp/Documents/GitHub/Tesis/data/low/"
fpath = '/Users/falconlab/Documents/GitHub/Tesis/data/low/'
fname = "file1_080124.mat"
#cutoff = [8/1000, 11/1000]

data = trainData(fpath, fname, 'Amplitude')
train, val, test = data.split_data('Amplitude')

import copy

train = utils.subsample(train, 2)
test = utils.subsample(test, 2)

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train.reshape(-1, 1))
test_scaled = scaler.transform(test.reshape(-1, 1))

test = copy.deepcopy(test_scaled)

x_train, y_train = ut2.create_sequences(train_scaled[:], 1000, 1)
x_test, y_test = ut2.create_sequences(test_scaled, 1000, 1)

x_train = torch.from_numpy(x_train.astype(np.float32)).squeeze()


0         0.006094
1         0.006132
2         0.006056
3         0.006018
4         0.006170
            ...   
599995    0.007766
599996    0.007741
599997    0.007761
599998    0.007748
599999    0.007755
Name: Amplitude, Length: 600000, dtype: float64
0         0.006094
1         0.006132
2         0.006056
3         0.006018
4         0.006170
            ...   
599995    0.007766
599996    0.007741
599997    0.007761
599998    0.007748
599999    0.007755
Name: Amplitude, Length: 600000, dtype: float64
0         0.006094
1         0.006132
2         0.006056
3         0.006018
4         0.006170
            ...   
599995    0.007766
599996    0.007741
599997    0.007761
599998    0.007748
599999    0.007755
Name: Amplitude, Length: 600000, dtype: float64
0         0.006094
1         0.006132
2         0.006056
3         0.006018
4         0.006170
            ...   
599995    0.007766
599996    0.007741
599997    0.007761
599998    0.007748
599999    0.007755
Name: Amplitude, Len

## modelo

In [3]:
in_size = x_train.shape[-1]
out_size = 1
hid_size = 3
num_layers = 1


model = SimpleRNN(in_size, hid_size, out_size, num_layers)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [5]:
filename = 'earlystop_187.pth'
utils.resume(model, optimizer, filename)

## **50 steps**

In [6]:
steps = 50
x_test , y_test = ut2.create_sequences(test, 1000, steps)
x_test = torch.from_numpy(x_test.astype(np.float32)).squeeze()
y_test = torch.from_numpy(y_test.astype(np.float32)).squeeze()


In [7]:
test50 = ut2.rollingWindowPrediction(model, x_test, steps)
np.savetxt("test50_RNN.csv", test50, delimiter=",")

100%|██████████| 43951/43951 [05:08<00:00, 142.43it/s]


In [8]:
mse, mape, r2, rmse = er.calculate_errors(test50, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.09710110882706069
Mean Absolute Percentage Error: 1.376685290497444
R2 Score: -8.477420081987562
Root Mean Squared Error: 0.31012731976279934


## **100 steps**

In [9]:
steps = 100
x_test , y_test = ut2.create_sequences(test, 1000, steps)
x_test = torch.from_numpy(x_test.astype(np.float32)).squeeze()
y_test = torch.from_numpy(y_test.astype(np.float32)).squeeze()

In [10]:
test100 = ut2.rollingWindowPrediction(model, x_test, steps)
np.savetxt("test100_RNN.csv", test100, delimiter=",")

100%|██████████| 43901/43901 [10:26<00:00, 70.08it/s]


In [11]:
mse, mape, r2, rmse = er.calculate_errors(test100, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.06549347928686731
Mean Absolute Percentage Error: 1.0189720192590013
R2 Score: -5.738326952086253
Root Mean Squared Error: 0.24241807593182435


## **500 steps**

In [12]:
steps = 500
x_test , y_test = ut2.create_sequences(test, 1000, steps)
x_test = torch.from_numpy(x_test.astype(np.float32)).squeeze()
y_test = torch.from_numpy(y_test.astype(np.float32)).squeeze()

In [13]:
test500 = ut2.rollingWindowPrediction(model, x_test, steps)
np.savetxt("test500_RNN.csv", test500, delimiter=",")

100%|██████████| 43501/43501 [1:10:04<00:00, 10.35it/s]


In [14]:
mse, mape, r2, rmse = er.calculate_errors(test500, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.04472074623771752
Mean Absolute Percentage Error: 0.7980173659486399
R2 Score: -4.373742639492199
Root Mean Squared Error: 0.19971637431062197


## **1000 steps**

In [15]:
steps = 1000
x_test , y_test = ut2.create_sequences(test, 1000, steps)
x_test = torch.from_numpy(x_test.astype(np.float32)).squeeze()
y_test = torch.from_numpy(y_test.astype(np.float32)).squeeze()

In [16]:
test1000 = ut2.rollingWindowPrediction(model, x_test, steps)
np.savetxt("test1000_RNN.csv", test1000, delimiter=",")

100%|██████████| 43001/43001 [2:19:18<00:00,  5.14it/s]  


In [17]:
mse, mape, r2, rmse = er.calculate_errors(test1000, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 0.06242700525649339
Mean Absolute Percentage Error: 0.9300775890304755
R2 Score: -7.131219953915073
Root Mean Squared Error: 0.2369488930861443
